In [8]:
###安装相关包###
!pip install pulp -i https://pypi.tuna.tsinghua.edu.cn/simple --user

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [11]:
###导入相关包###
import pulp
import numpy as np
from pprint import pprint

In [12]:
# 定义最优化问题
def sales_max(costs, x_max, y_max):
    row = len(costs)
    col = len(costs[0])
    print(f'row:{row} col:{col}')
    # 定义初始问题
    prob = pulp.LpProblem('sales_max', sense=pulp.LpMaximize)
    # 定义相关变量
    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]
    # 递归展开列表
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    print('var=', var)
    print('flatten=', flatten)
    print('var[1]', var[1])
    # 定义目标函数 
    prob += pulp.lpDot(flatten(var), costs.flatten())
    # 定义约束条件
    for i in range(row):
        prob += pulp.lpSum(var[i]) <= x_max[i]
    for j in range(col):
        prob += pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j]

    prob.solve()
    result = {'objective':pulp.value(prob.objective), \
              'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}
    return result

In [13]:
# 成本
costs = np.array([[500, 550, 630, 1000, 800, 700],
                  [800, 700, 600, 950, 900, 930],
                  [1000, 960, 840, 650, 600, 700],
                  [1200, 1040, 980, 860, 880, 780]])
# 最大产量
max_plant = [76, 88, 96, 40]
# 最大耕种
max_cultivation = [42, 56, 44, 39, 60, 59]

In [14]:
# 使用线性规划
result = sales_max(costs, max_plant, max_cultivation)
# 输出结果
print(f'最大值为{result["objective"]}')
print('各变量的取值为：')
pprint(result['var'])

row:4 col:6
var= [[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]
flatten= <function sales_max.<locals>.<lambda> at 0x7f115af49830>
var[1] [x10, x11, x12, x13, x14, x15]
最大值为284230.0
各变量的取值为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
 [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
 [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
